Nama : Rika Ajeng Finatih

NIM : 121450036

Kelas : Pengenalan Pola RA

# ***Mel Frequency Cepstral Coefficients* (MFCC)**

## Import Library

In [5]:
import numpy as np
from scipy.fftpack import dct

## Sinyal dan Paramater

In [6]:
x = np.array([1, 2, 0, -1, -2, -1, 1, 2])  # Sinyal
fs = 8  # Frekuensi sampling
frame_size = 4  # Ukuran frame (4 sampel)
frame_step = 2  # Overlap 2 sampel
alpha = 0.95  # Pre-emphasis coefficient
num_filters = 2  # Jumlah mel filter

## **Langkah 1**: Pre-emphasis

Formula pre-emphasis:

$$y[t] = x[t] - α⋅x(t−1)$$

Keterangan:


*   $y(t)$ adalah nilai sinyal pada waktu n setelah pre-emphasis.
*   $x[t]$ adalah nilai sinyal asli pada waktu n.
*   $x(t−1)$ adalah nilai sinyal pada waktu sebelumnya.
*   $α$ adalah faktor pre-emphasis yang biasanya berada dalam rentang $0<α≤1$, dengan nilai $α=0.97$.



In [7]:
# Fungsi untuk menghitung pre-emphasis
def pre_emphasis(signal, alpha):
    return np.append(signal[0], signal[1:] - alpha * signal[:-1])

pre_emphasized = pre_emphasis(x, alpha)

# Cetak Output
print("Pre-emphasized Signal:", pre_emphasized)

Pre-emphasized Signal: [ 1.    1.05 -1.9  -1.   -1.05  0.9   1.95  1.05]


Sehingga diperoleh nilai pre-emphasis sebagai berikut:

$$y[t] = [1, 1.05, -1.9, -1, -1.05, 0.9, 1.95, 1.05]$$

## **Langkah 2**: Framing (Frame Blocking)

Memecah sinyal kedalam sebuah segmen-segmen kecil.

In [8]:
# Fungsi untuk menghitung farming
def framing(signal, frame_size, frame_step):
    signal_length = len(signal)
    num_frames = int(np.ceil((signal_length - frame_size) / frame_step)) + 1
    pad_signal_length = num_frames * frame_step + frame_size
    pad_signal = np.append(signal, np.zeros(pad_signal_length - signal_length))
    frames = np.zeros((num_frames, frame_size))
    for i in range(num_frames):
        start_idx = i * frame_step
        frames[i, :] = pad_signal[start_idx:start_idx + frame_size]
    return frames

frames = framing(pre_emphasized, frame_size, frame_step)

# Cetak Output
print("Frames:\n", frames)

Frames:
 [[ 1.    1.05 -1.9  -1.  ]
 [-1.9  -1.   -1.05  0.9 ]
 [-1.05  0.9   1.95  1.05]]


Frames diperoleh dengan parameter ukuran frame 4 sampel, overlap 2 sampel, dan stride 2 sampel, sehingga hasilnya adalah sebagai berikut:


*   Frame 1 : $[1, 1.05, -1.9, -1]$
*   Frame 2 : $[-1.9, -1, -1.05, 0.9]$
*   Frame 3 : $[-1.05, 0.9, 1.95, 1.05]$





## **Langkah 3**: Windowing (Hamming)

Formula Hamming:
$$w(n) = 0.54 - 0.46 \cdot \cos\left( \frac{2 \pi n}{N-1} \right)
$$

Keterangan:



*   $w(n)$ adalah nilai window pada indeks $𝑛$ (nilai yang akan diterapkan pada sinyal).
*   $N$ adalah panjang window (jumlah sampel dalam window).
*   $n$ adalah indeks sampel, dengan $𝑛
=
0
,
1
,
2
,
…
,
𝑁
−
1
n=0,1,2,…,N−1.$



Kemudian, hitung nilai windowing dengan formula sebagai berikut.

$$x'(n) = x(n).w(n)$$




In [9]:
# Fungsi untuk menghitung windowing(hamming)
def hamming_window(frames):
    window = np.hamming(frames.shape[1])
    return frames * window

windowed_frames = hamming_window(frames)

# Cetak Output
print("Windowed Frames:\n", windowed_frames)

Windowed Frames:
 [[ 0.08    0.8085 -1.463  -0.08  ]
 [-0.152  -0.77   -0.8085  0.072 ]
 [-0.084   0.693   1.5015  0.084 ]]


Dengan mengaplikasikan hasil frames sebelumnya ke dalam windowing menggunakan fungsi Hamming, diperoleh nilai sebagai berikut:



*   Windowed Frame 1: $[0.08, 0.8085, -1.463, -0.08]$
*   Windowed Frame 2: $[-0.152, -0.77   -0.8085, 0.072]$
*   Windowed Frame 3: $[-0.084, 0.693, 1.5015, 0.084]$



## **Langkah 4**: FFT dan Power Spectrum

Formula FFT:
$$
X(k) = \sum_{n=0}^{N-1} x(n) \cdot e^{-j \cdot 2\pi \cdot \frac{k \cdot n}{N}}
$$

Keterangan:


*   $X(k)$ adalah nilai Fourier pada frekuensi $k$.
*   $x(n)$ adalah nilai sinyal pada waktu $n$.
*   $N$ adalah panjang sinyal.
*   $k$ adalah indeks frekuensi.
*   $j$ adalah bilangan imajiner $(j=\sqrt(-1))$



Kemudian menghitung power spectrum dengan formula berikut:

$$P(f) = \frac{1}{N} \left| X(f) \right|^2
$$

Keterangan:



*   $P(f)$  adalah power spectrum pada frekuensi $f$
*   $X(f)$ adalah spektrum sinyal setelah Fourier Transform
*   $N$ adalah jumlah total sampel dalam sinyal atau panjang FFT yang digunakan.



In [21]:
# Fungsi untuk menghitung FFT dan Power Spectrum
def power_spectrum(frames, fft_size):
    fft_frames = np.fft.rfft(frames, n=fft_size)  # FFT
    power_frames = (np.abs(fft_frames) ** 2) / fft_size  # Power Spectrum
    return fft_frames, power_frames

# Data sampel windowed_frames yang diperoleh sebelumnya
windowed_frames = np.array([
    [0.08, 0.8085, -1.463, -0.08],
    [-0.152, -0.77, -0.8085, 0.072],
    [-0.084, 0.693, 1.5015, 0.084 ],
])

fft_size = 4  # Panjang FFT
fft_frames, power_frames = power_spectrum(windowed_frames, fft_size)

# Cetak hasil FFT
print("FFT Frames:\n", fft_frames)

FFT Frames:
 [[-0.6545+0.j      1.543 -0.8885j -2.1115+0.j    ]
 [-1.6585+0.j      0.6565+0.842j  -0.2625+0.j    ]
 [ 2.1945+0.j     -1.5855-0.609j   0.6405+0.j    ]]


In [22]:
# Cetak Power Spectrum
print("\nPower Spectrum:\n", power_frames)


Power Spectrum:
 [[0.10709256 0.79257031 1.11460806]
 [0.68765556 0.28498906 0.01722656]
 [1.20395756 0.72117281 0.10256006]]


Dengan menggunakan FFT dengan panjang
$𝑁 = 4$, diperoleh nilai power spectrum sebagai berikut:

*   Power Spectrum 1: $[0.10709256, 0.79257031, 1.11460806]$
*   Power Spectrum 2: $[0.68765556, 0.28498906, 0.01722656]$
*   Power Spectrum 3: $[1.20395756, 0.72117281, 0.10256006]$



## **Langkah 5**: Mel Filter Bank

Formula fungsi mel:
$$E_m = \sum_{k=0}^{K-1} |X_k|^2 \cdot H_m(k)$$

Keterangan:


*   $|X_k|^2$ adalah Power spectrum dari frame.
*   $ H_m(k)$ adalah Respons Mel filter.



Dimana;
$$H_m(k) =
\begin{cases}
0 & \text{jika } k < f(m-1) \text{ atau } k > f(m+1) \\
\frac{k - f(m-1)}{f(m) - f(m-1)} & \text{jika } f(m-1) \leq k < f(m) \\
\frac{f(m+1) - k}{f(m+1) - f(m)} & \text{jika } f(m) \leq k \leq f(m+1)
\end{cases}$$

In [12]:
# Fungsi untuk menghitung mel
def mel_filter_bank(power_frames, num_filters, fft_size):
    filters = np.zeros((num_filters, fft_size // 2 + 1))
    filters[0, 1:3] = [1, 0.5]  # Filter 1
    filters[1, 1:3] = [0.5, 1]  # Filter 2
    energies = np.dot(power_frames, filters.T)
    return energies

mel_energies = mel_filter_bank(power_frames, num_filters, fft_size)

# Cetak Output
print("Mel Filter Energies:\n", mel_energies)

Mel Filter Energies:
 [[1.34987434 1.51089322]
 [0.29360234 0.15972109]
 [0.77245284 0.46314647]]


Dengan menggunakan filter 1: $[1, 0.5]$ dan filter 2: $[0.5, 1]$ diperoleh mel filter energies dengan menghitung energi untuk setiap frame sebagai berikut:



*   Mel Filter Energies 1: $[1.34987434, 1.51089322]$
*   Mel Filter Energies 2: $[0.29360234, 0.15972109]$
*   Mel Filter Energies 3: $[0.77245284, 0.46314647]$



## **Langkah 6**: Log Energies

Formula log energies:

$$E_{\text{log}}(f) = \log(P(f) + \epsilon)
$$

Keterangan:



*   $E_{\text{log}}(f)$ adalah logaritma energi pada frekuensi $f$.

*   $P(f)$ adalah power spectrum pada frekuensi $f$.
*   $\epsilon$ adalah nilai kecil yang ditambahkan untuk menghindari logaritma dari nol.



In [13]:
# Fungsi untuk menghitung log energies
log_energies = np.log(mel_energies + 1e-8)  # Tambahkan epsilon untuk menghindari log(0)

# Cetak Output
print("Log Energies:\n", log_energies)

Log Energies:
 [[ 0.30001152  0.41270102]
 [-1.22552897 -1.83432609]
 [-0.2581843  -0.76971191]]


Diperoleh log energies untuk setiap frame sebagai berikut:



*   Log Energies 1: $[ 0.30001152, 0.41270102]$
*   Log Energies 2: $[-1.22552897, -1.83432609]$
*   Log Energies 3: $[-0.2581843, -0.76971191]$



## **Langkah 7**: DCT

Formula Discrete Consine Transform (DCT) tipe II yagn digunakan untuk menghitung koefisien MFCC sebagai berikut:
$$C_k = \sum_{n=0}^{N-1} x_n \cdot \cos\left(\frac{\pi}{N} \cdot \left(n + \frac{1}{2}\right) \cdot k\right)$$

Keterangan:



*   $C_k$ adalah koefisien DCT ke-$k$
*   $x_n$ adalah nilai input (log energi pada filter bank mel) untuk indeks $n$.
*   $N$ adalah jumlah elemen dalam input $x$.
*   $k$ adalah indeks koefisien DCT $(0 ≤ k < N)$.





In [14]:
# Fungsi untuk menghitung DCT
def dct_transform(log_energies, num_cepstral):
    return dct(log_energies, type=2, axis=1, norm='ortho')[:, :num_cepstral]

num_cepstral = 2  # Ambil 2 koefisien pertama
mfcc = dct_transform(log_energies, num_cepstral)


# Cetak Output
print("MFCC:\n", mfcc)

MFCC:
 [[ 0.50396387 -0.07968351]
 [-2.16364426  0.43048457]
 [-0.72683238  0.36170464]]


Terakhir, diperolehlah nilai DCT dengan mengambil 2 koefisien pertama sebagai berikut:



*   MFCC Frame 1: $[ 0.50396387, -0.07968351]$
*   MFCC Frame 2: $[-2.16364426, 0.43048457]$
*   MFCC Frame 3: $[-0.72683238, 0.36170464]$



